In [1]:
import math
import numpy as np
import os
from collections import defaultdict
from collections import Counter
import tensorflow as tf 
from sklearn import preprocessing

In [2]:
data_load = np.load('processed_array.gz.npy')

In [22]:
def select_qid(data, qid1, qid2):
    # Select inst for specified qid, eg '116'
    
    data_sub = []
    for i in range(len(data)-1):
        if data[i][1] == qid1 or data[i][1] == qid2:
            data_sub.append(data[i])
    print('Qid:', qid, np.asarray(data_sub).shape)
    return data_sub

def normalise_feats(data_sub):
    # Standardise features (per column) by -mean and /SD
    # Replace rel and qid with unnormalised
    
    data_sub = np.asarray(data_sub)
    data_sub_norm = preprocessing.scale(data_sub, axis=1)
    data_sub_norm[:, 0] = data_sub[:, 0]
    data_sub_norm[:, 1] = data_sub[:, 1]
    return data_sub_norm

def generate_pairs(data_sub):
    # Generate pairs of non-matching relievence. Higher relevence 1st, lower 2nd
    
    pair_sets= []
    pair_y= []
    for i in data_sub:
        for j in data_sub:
            if i[1]== j[1]:
                pair_sets.append((i, j))
                if i[0]> j[0]:
                    pair_y.append([1])
                if i[0]== int(j[0]):
                    pair_y.append([0.5])
                if i[0]< j[0]:
                    pair_y.append([0])
                
    return np.asarray(pair_sets), np.asarray(pair_y)

In [24]:
data_166 = select_qid(data_load, '166', '142')
data_166_n = normalise_feats(data_166)
x_116, y_166 = generate_pairs(data_166_n)
len(x_116)

Qid: Tensor("Cast_2:0", shape=(), dtype=int32) (214, 138)


/Users/jamesshields/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype <U16 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


24698

In [35]:
x_pair = tf.placeholder("float", [None, 2, 138])
y_gold = tf.placeholder("float", [None, 1])

x_feat1 = tf.slice(x_pair, [0, 0, 2], [-1, 1, 136])
x_feat2 = tf.slice(x_pair, [0, 1, 2], [-1, 1, 136])
x_feat1 = tf.reshape(x_feat1, [-1, 136])
x_feat2 = tf.reshape(x_feat2, [-1, 136])
qid = tf.cast(x_pair[0,0,1], dtype='int32')
d_qid = qid- 1

d_in = 136
d_hidden = 500
d_out = 1
n_qid = 166

W1 = tf.Variable(tf.random_normal([n_qid, d_in, d_hidden], mean= 0.01, stddev= 0.1))
b1 = tf.Variable(tf.random_normal([n_qid, d_hidden], mean= 0.01, stddev= 0.1))
W2 = tf.Variable(tf.random_normal([n_qid, d_hidden, d_out], mean= 0.01, stddev= 0.1))
b2 = tf.Variable(tf.random_normal([n_qid, d_out], mean= 0.01, stddev= 0.1))

a1_i = tf.matmul(x_feat1, W1[d_qid, :, :])+ b1[d_qid, :]
z1_i = tf.sigmoid(a1_i)
o_i = tf.matmul(z1_i, W2[d_qid, :, :])+ b2[d_qid, :]

a1_j = tf.matmul(x_feat2, W1[d_qid, :, :])+ b1[d_qid, :]
z1_j = tf.sigmoid(a1_j)
o_j = tf.matmul(z1_j, W2[d_qid, :, :])+ b2[d_qid, :]

o_ij = o_i- o_j
P_ij = tf.exp(o_ij)/ (1+ tf.exp(o_ij))

#C_ij = -P_ij* tf.log(P_ij) - (1- P_ij)* tf.log(1- P_ij)
cross_entropy = -tf.reduce_sum(y_gold* tf.log(tf.clip_by_value(P_ij, 1e-10,1.0)))
optimiser = tf.train.GradientDescentOptimizer(0.015).minimize(cross_entropy)

prediction = tf.round(P_ij * 2) / 2
mistakes = tf.not_equal(y_gold, prediction)
accuracy = 1- tf.reduce_mean(tf.cast(mistakes, tf.float32))

In [6]:
# Variable dev
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    init_dict= {x_pair: x_116[:100],
                y_gold: y_166[:100]}

    print(sess.run(qid, feed_dict= init_dict))

166


In [38]:
# Train and test
batch_sz= 30
iter_= int(x_116.shape[0]/ batch_sz)
epoch= 10

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    init_dict= {x_pair: x_116[:100],
                y_gold: y_166[:100]}
    print('Init accuracy:', (sess.run(accuracy, feed_dict= init_dict)))
    
    for e in range(epoch):
        e_loss= 0
        for i in range(iter_):
            i_loss= 0
            sta= i* batch_sz
            end= (i+ 1)* batch_sz
            iter_dict= {x_pair: x_116[sta: end],
                        y_gold: y_166[sta: end]}
            sess.run(optimiser, feed_dict= iter_dict)
            e_loss+= sess.run(cross_entropy, feed_dict= iter_dict)
        if e% (epoch/ 10)== 0:
            print('Epoch', e, 'loss:', e_loss)
        
    print('Trained accuracy:', (sess.run(accuracy, feed_dict= init_dict)))
    
    test_dict= {x_pair: x_116[100:1000],
                y_gold: y_166[100:1000]}    
    print('Test accuracy:' ,(sess.run(accuracy, feed_dict= test_dict)))

Init accuracy: 0.33
Epoch 0 loss: 5935.0181511
Epoch 1 loss: 6840.7761392
Epoch 2 loss: 6996.24825177
Epoch 3 loss: 7080.07737792
Epoch 4 loss: 7121.56140922
Epoch 5 loss: 7181.3146451
Epoch 6 loss: 7235.83302364
Epoch 7 loss: 7282.1647341
Epoch 8 loss: 7338.92982009
Epoch 9 loss: 7369.24999267
Trained accuracy: 0.34
Test accuracy: 0.47
